In [93]:
from dotenv import load_dotenv
import os
from google import genai
from google.genai import types
# from IPython.display import Markdown
import markdown
import pymupdf

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.sentence_transformer import (SentenceTransformerEmbeddings)
from langchain_community.vectorstores import SQLiteVec

load_dotenv()

True

In [91]:
client = genai.Client(api_key=os.getenv("GEMINI_API"))
embedding_function = SentenceTransformerEmbeddings(model_name='BAAI/bge-m3')
spliter = RecursiveCharacterTextSplitter( chunk_size=2048, chunk_overlap=20,separators=[".","\n"])

In [ ]:
PDF_path = r"Documents\Python C-Dac (1 - June - 2025).pdf"
doc = pymupdf.open(PDF_path)
text_to_chat = " ".join([page.get_text().replace("\n","") for page in doc])
chunks = spliter.split_text(text_to_chat)
db = SQLiteVec.from_texts(texts=chunks,embedding=embedding_function, table="state_union",db_file="vec.db",)

In [ ]:
user_query = "History of Python"
data = db.similarity_search(user_query)

text_to_chat = ""
for i in data:
    text_to_chat += i.page_content

In [106]:
def call_llm(user_question):
    response = client.models.generate_content(
        model="gemini-2.5-flash-lite",
        config = types.GenerateContentConfig(
            system_instruction=f"You are an expert in Python. You must answer questions only using the provided context. If a question is outside the provided context, respond strictly with: 'It is not in the PDF'. Do not add extra explanations. context text ```{text_to_chat} ``` "),
        contents=user_question,
    )

    return response

In [107]:
response =call_llm(user_query)


In [108]:
response.text

'Guido van Rossum began developing Python in December 1989 as a hobby project while working at the National Research Institute for Mathematics and Computer Science in the Netherlands. He was inspired by ABC, a general-purpose programming language, and set out to create a more intuitive and easy-to-learn language with similar features. The first version of Python, version 0.9.0, was released in February 1991. Over the next few years, Guido continued to work on Python and released several major versions, with Python 2.0 being released in 2000. In 2008, development of Python 3.0 began, with the goal of addressing some of the design flaws in Python 2.x and improving the language overall. Python 3.0 was released in 2008, and since then, all new development efforts have been focused on Python 3.x.'